# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, но при этом качество модели линейной регрессии не ухудшилось.

## Содержание<a name="Содержание"></a>

1. [Знакомство с данными](#1.-Знакомство-с-данными)
2. [Теоретическое обоснование метода шифрования](#2.-Теоретическое-обоснование-метода-шифрования)
	1. [Задача линейной регрессии](#Задача-линейной-регрессии)
	2. [Умножение на обратимую матрицу](#Умножение-на-обратимую-матрицу)
3. [Алгоритм преобразования](#3.-Алгоритм-преобразования)
4. [Проверка алгоритма](#4.-Проверка-алгоритма)
	1. [Незакодированные данные: обучение и проверка модели](#Незакодированные-данные:-обучение-и-проверка-модели)
	2. [Матрица кодирования](#Матрица-кодирования)
	3. [Закодированные данные: обучение и проверка модели](#Закодированные-данные:-обучение-и-проверка-модели)
5. [Вывод](#5.-Вывод)

## 0. Импорт библиотек

In [1]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
SEED = 42

## 1. Знакомство с данными<a name="1.-Знакомство-с-данными"></a>

Сохраним датафрейм в переменную `insurance` и посмотрим на первые строки из него.

In [3]:
insurance = pd.read_csv('insurance.csv')
insurance.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Каждый объект в таблице содержит информацию об одном пользователе:
- пол,
- возраст,
- размер зарплаты,
- количество членов семьи,
- число страховых выплат за последние пять лет.

Последний признак — целевой.

Посмотрим на общую информацию о данных.

In [4]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Нам повезло, в данных нет пропусков. 

Для нашей задачи такого знакомства достаточно — не будем исправлять названия столбцов, искать выбросы и дупликаты, приводить признаки к нужным типам.

[К содержанию](#Содержание)

## 2. Теоретическое обоснование метода шифрования<a name="2.-Теоретическое-обоснование-метода-шифрования"></a>

### Задача линейной регрессии<a name="Задача-линейной-регрессии"></a>

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Умножение на обратимую матрицу<a name="Умножение-на-обратимую-матрицу"></a>

При умножении признаков на обратимую матрицу качество линейной регрессии **не изменится.**

**Обоснование:** 

Обозначим новую матрицу признаков как $X'$:

$$ X' = XP $$

Тогда новый вектор весов $w'$ примет такой вид:

$$ w' = ((XP)^T (XP))^{-1} (XP)^T y $$

Найдем новые предсказания $a'$:

$$ a' = X'w' = $$ 
$$ $$
$$ = (XP)((XP)^T (XP))^{-1} (XP)^T y =$$
$$ $$
$$ = [по⠀свойству⠀транспонирования⠀произведения⠀матриц⠀(AB)^T=B^T A^T] $$
$$ XP(P^T X^T XP)^{-1} P^T X^T y =$$ 
$$ $$
$$ = [по⠀ассоциативности⠀и⠀свойству⠀обратной⠀матрицы⠀произведения⠀(AB)^{-1}=B^{-1}A^{-1}] $$
$$ XP P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y =$$
$$ $$
$$ = [по⠀определению⠀обратной⠀матрицы] $$
$$ XE (X^T X)^{-1} E X^T y =$$
$$ $$
$$ = X (X^T X^{-1}) X^T y = Xw$$

Новые предсказания равны старым, что и требовалось доказать.

[К содержанию](#Содержание)

## 3. Алгоритм преобразования<a name="3.-Алгоритм-преобразования"></a>

1. С помощью `np.random.rand()` создадим квадратную матрицу A с размерностью равной числу признаков в исходных данных.

2. Методом `np.linalg.inv()` проверим матрицу на обратимость. Если метод не вернул ошибку, то матрица обратимая, в противном случае — нет.

3. Умножим матрицу признаков X на А. Результат и будет закодированными данными пользователей.

**Обоснование**

Представленный алгоритм вернет зашифрованные данные пользователей. При этом он не повлияет на работу модели — мы доказали это в предыдущем пункте.

[К содержанию](#Содержание)

## 4. Проверка алгоритма<a name="4.-Проверка-алгоритма"></a>

Выделим из датафрейма `insurance` признаки и целевой признак.

In [5]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

Разобьем данные на обучающую и тестовую выборки и проверим их размеры.

In [6]:
features_train, features_test, target_train, target_test = \
        train_test_split(features, target, test_size=0.25, random_state=SEED)

In [7]:
print('\033[1m' + 'Размеры выборок' + '\033[0m')
print('Обучающая:', features_train.shape)
print('Тестовая:', features_test.shape)

Размеры выборок
Обучающая: (3750, 4)
Тестовая: (1250, 4)


### Незакодированные данные: обучение и проверка модели<a name="Незакодированные-данные:-обучение-и-проверка-модели"></a>

Обучим модель на незакодированных данных, получим предсказания на тестовой выборке и посмотрим на их качество.

In [8]:
model = LinearRegression()
model.fit(features_train, target_train)

preds_initial = model.predict(features_test)

Для проверки качества модели будем использовать коэффициент детерминации. Он показывает, насколько хорошо рассматриваемая модель объясняет дисперсию целевой переменной.

In [9]:
r2_initial = r2_score(target_test, preds_initial)

print('Коэффициент детерминации модели, обученной на исходных данных:', r2_initial)

Коэффициент детерминации модели, обученной на исходных данных: 0.4254778540696311


Коэффициент получился ниже 50%, то есть модель плохо понимает данные 😬 Но сейчас мы не будем разбираться с этим.

### Матрица кодирования<a name="Матрица-кодирования"></a>

Теперь создадим матрицу, которой будем кодировать признаки, и проверим ее на вырожденность.

In [10]:
np.random.seed(SEED)
A = np.random.rand(4,4)
np.linalg.inv(A)

array([[-0.32076901, -0.12766508,  0.06141427,  1.32518674],
       [ 0.35151041, -1.88500014,  1.65560045, -1.0003883 ],
       [ 1.14080312,  1.3467702 , -2.0407373 ,  0.70794071],
       [-0.08202687,  1.42666425, -0.17238177, -0.10600441]])

Обратная матрица создалась, значит, A не вырождена. Домножим на нее матрицу признаков, а затем создадим новые, состоящие из закодированных признаков обучающую и тестовую выборки.

In [11]:
features_enc = features.values @ A

features_enc_train, features_enc_test, target_enc_train, target_enc_test = \
        train_test_split(features_enc, target, test_size=0.25, random_state=42)

In [12]:
print('\033[1m' + 'Размеры выборок' + '\033[0m')
print('Обучающая:', features_enc_train.shape)
print('Тестовая:', features_enc_test.shape)

Размеры выборок
Обучающая: (3750, 4)
Тестовая: (1250, 4)


### Закодированные данные: обучение и проверка модели<a name="Закодированные-данные:-обучение-и-проверка-модели"></a>

Переобучим модель на закодированных данных и посмотрим на качество новых предсказаний.

In [13]:
model.fit(features_enc_train, target_enc_train)

preds_enc = model.predict(features_enc_test)
r2_enc = r2_score(target_enc_test, preds_enc)

print('R2-метрика модели, обученной на закодированных данных:', r2_enc)

R2-метрика модели, обученной на закодированных данных: 0.42547785406900973


Вспомним результат модели на необработанных данных.

In [14]:
print('R2-метрика модели, обученной на исходных данных:', r2_initial)

R2-метрика модели, обученной на исходных данных: 0.4254778540696311


Метрики различаются после десятого знака за запятой. Спишем это небольшое различие на особенности хранения данных в Питоне.

[К содержанию](#Содержание)

## 5. Вывод <a name="5.-Вывод"></a>

Главная задача проекта — разработать метод кодирования данных, который не повлияет на качество моделей машинного обучения. Мы справились с ней! 

Для решения задачи мы теоретически доказали, что предсказания линейной регрессии не изменятся, если подаваемые ей признаки будут умножены на обратимую матрицу. На основе этого свойства мы создали алгоритм кодирования данных. Затем мы проверили его на реальных данных и получили подтверждающий теорию результат.

[К содержанию](#Содержание)